# Preprocessing IMDB Dataset
# ML Project Phase 2 Submission



Group Members: -

Name: - Priyanka R Wakte

Student Id :- 1001920878

Email Id :- prw0878@mavs.uta.edu



# Sentiment Analysis on IMDB Reviews


In [8]:
# Vocabulary: All words used, starting by the most frequent
with open('C:/Users/priya/OneDrive - University of Texas at Arlington/Documents/Machine Learning/aclImdb/imdb.vocab',encoding = 'cp850') as f:
    vocab = [word.rstrip() for word in f]
    # Keep only most frequent 5000 words
    vocab = vocab[:5000]
    print('%d words in vocabulary' % (len(vocab),))

5000 words in vocabulary


In [9]:
import pandas as pd
import numpy as np
import os
import csv 
import numpy as np
from pandas import DataFrame, read_csv
from nltk.corpus import stopwords
from matplotlib import pyplot
import seaborn as sns
import nltk
import re
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer 
from nltk.stem import PorterStemmer 
from nltk.stem import LancasterStemmer
from nltk.stem import WordNetLemmatizer

from keras.datasets import imdb
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence
import tensorflow as ft

nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')
stop_words=set(stopwords.words('english'))


[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\priya\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\priya\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\priya\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [10]:
def tokenize(t):
    t = t.lower()

    t = re.sub("\\s", " ", t) 

    t = re.sub("[^a-z' ]", "", t) 

    
    w_bow = word_tokenize(t)
    bow = []
    for k in w_bow:
        if k not in stop_words:
            bow.append(k)
    
    pstem = PorterStemmer()
    sstem = SnowballStemmer(language='english')
    lstem = LancasterStemmer()
    for word in bow:
        
        pstem_text=pstem.stem(word)
        sstem_text=sstem.stem(word)
        lstem_text=lstem.stem(word)

    final_bow=[]
    
    lemmatizer = WordNetLemmatizer()
    for word in bow:
      final_bow.append(lemmatizer.lemmatize(word))

    return final_bow

In [11]:
def load_dt(dirname):
    a, b = [], []
    
    for a_val, b_label in enumerate(['neg', 'pos']):
        b_dir = os.path.join(dirname, b_label)
        for filename in os.listdir(b_dir):
            filepath = os.path.join(b_dir, filename)
            # print('\r' + fpath + '   ', end='')
            with open(filepath,encoding="UTF-8" ) as f:
                bow = tokenize(f.read())
            a.append(bow)
            b.append(a_val) 
    return a, b

In [12]:
atrain, btrain = load_dt('C:/Users/priya/OneDrive - University of Texas at Arlington/Documents/Machine Learning/aclImdb/train/')


In [5]:
text="Bromwell High is a cartoon comedy. It ran at the same time as some other programs about school life, such as 'Teachers'. My 35 years in the teaching profession lead me to believe that Bromwell High's satire is much closer to reality than is 'Teachers'. The scramble to survive financially, the insightful students who can see right through their pathetic teachers' pomp, the pettiness of the whole situation, all remind me of the schools I knew and their students. When I saw the episode in which a student repeatedly tried to burn down the school, I immediately recalled ......... at .......... High. A classic line: INSPECTOR: I'm here to sack one of your teachers. STUDENT: Welcome to Bromwell High. I expect that many adults of my age think that Bromwell High is far fetched. What a pity that it isn't!"

bow=tokenize(text)
print(bow)


['bromwell', 'high', 'cartoon', 'comedy', 'ran', 'time', 'program', 'school', 'life', "'teachers", "'", 'year', 'teaching', 'profession', 'lead', 'believe', 'bromwell', 'high', "'s", 'satire', 'much', 'closer', 'reality', "'teachers", "'", 'scramble', 'survive', 'financially', 'insightful', 'student', 'see', 'right', 'pathetic', 'teacher', "'", 'pomp', 'pettiness', 'whole', 'situation', 'remind', 'school', 'knew', 'student', 'saw', 'episode', 'student', 'repeatedly', 'tried', 'burn', 'school', 'immediately', 'recalled', 'high', 'classic', 'line', 'inspector', "'m", 'sack', 'one', 'teacher', 'student', 'welcome', 'bromwell', 'high', 'expect', 'many', 'adult', 'age', 'think', 'bromwell', 'high', 'far', 'fetched', 'pity', "n't"]


In [4]:
def imdb_data_preprocess(inpath, outpath="./", name="imdb_tr.csv", mix=False):
    global stoplist
    stoplist = set(stopwords.words("english"))
    review = []
    polarity = []

    

    for filename in os.listdir(inpath+"pos"):
        data = open(inpath+"pos/"+filename, 'r' , encoding="ISO-8859-1").read()
        data = re.sub("[^a-z' ]", "", data)
        data = re.sub("\\s", " ", data) 
        data = del_stpwrds(data, stoplist)
        
        review.append(data)
        polarity.append("Positive")
        

    for filename in os.listdir(inpath+"neg"):
        data = open(inpath+"neg/"+filename, 'r' , encoding="ISO-8859-1").read()
        data = re.sub("[^a-z' ]", "", data)
        data = re.sub("\\s", " ", data) 
        data = del_stpwrds(data, stoplist)
        
        review.append(data)
        polarity.append("Negative")
        

    Dataset = list(zip(review,polarity))

    if mix:
        np.random.shuffle(Dataset)

    df = pd.DataFrame(data = Dataset, columns=['review', 'polarity'])
    df.to_csv(outpath+name, index=False, header=True)
    pass


In [5]:
def del_stpwrds(sentence, stopwords):
    sentences = sentence.split()
    reswords  = [word for word in sentences if word.lower() not in stopwords]
    r = ' '.join(reswords)
    return r

In [6]:
imdb_data_preprocess(inpath='C:/Users/priya/OneDrive - University of Texas at Arlington/Documents/Machine Learning/aclImdb/train/', mix=True)

In [7]:
df = pd.read_csv('C:/Users/priya/OneDrive - University of Texas at Arlington/Documents/Machine Learning/aclImdb/imdb_tr.csv')
df

,review,polarity
0,one best movies 've ever seen good acting anks...,Positive
1,ver read im hompson e's hardboiled noir extrem...,Positive
2,film bad way say story weak outdated especiall...,Negative
3,hen inlaws viciously murdered gang thugs young...,Negative
4,ello laymates recently watched film first time...,Positive
...,...,...
24995,movie decide wanted couple subplots awhile mad...,Negative
24996,first installment notorious horror series pres...,Positive
24997,miss ark ngelbr br understand ever one likes f...,Positive
24998,know nime 've long became national phenomenon ...,Positive


In [10]:
X_val, y_val = load_dt('C:/Users/priya/OneDrive - University of Texas at Arlington/Documents/Machine Learning/aclImdb/test/')

In [11]:
def bow_onehot_vector(tokens):
    vector = [0] * len(vocab)
    for t in tokens:
        try:
            vector[vocab.index(t)] = 1
        except:
            pass  # ignore missing words
    return vector

In [15]:
from tqdm import tqdm #progress bar

X_bow_train = [bow_onehot_vector(x) for x in tqdm(X_train)] 
X_bow_val = [bow_onehot_vector(x) for x in tqdm(X_val)]


 39%|███▉      | 9736/25000 [00:31<00:59, 257.08it/s]


 77%|███████▋  | 19241/25000 [01:02<00:19, 295.90it/s]


100%|██████████| 25000/25000 [01:22<00:00, 304.74it/s]

 15%|█▍        | 3644/25000 [00:11<01:07, 314.17it/s]


 54%|█████▍    | 13616/25000 [00:43<00:33, 339.13it/s]


 92%|█████████▏| 23116/25000 [01:16<00:06, 312.24it/s]


100%|██████████| 25000/25000 [01:23<00:00, 300.58it/s]


In [16]:
from sklearn.naive_bayes import MultinomialNB

In [17]:
MNB = MultinomialNB()
MNB.fit(X_bow_train, y_train)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [18]:
from sklearn import metrics
predict_train_data = MNB.predict(X_bow_train)
accuracy_score1=metrics.accuracy_score(predict_train_data,y_train)
predict_test_data = MNB.predict(X_bow_val)
accuracy_score2 = metrics.accuracy_score(predict_test_data, y_val)

In [19]:
print(str('{:04.2f}'.format(accuracy_score1*100))+'%')
print(str('{:04.2f}'.format(accuracy_score2*100))+'%')

86.13%
84.50%
